<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/SBShim/LSTM_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install konlpy

In [83]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16182 sha256=36031db9997dc900bf543b5cf7d3afbd9d2571e7a5d3d70b7489de47f62916d6
  Stored in directory: /root/.cache/pip/wheels/b2/a1/b7/0d3b008d0c77cd57332d724b92cf7650b4185b493dc785f00a
Successfully built seqeval


In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

# 데이터
- 전처리 및 맞춤법 검사가 된 문장

In [53]:
data = pd.read_csv('/content/drive/MyDrive/공부/AIFFELTHON/shared/TUNiB/train_1000.csv')

In [54]:
data

,Unnamed: 0,class,label_sentence,spell_checked_conversation,binary_class
0,0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...,공격 대화
2,2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...,공격 대화
3,3,갈취 대화,1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...,공격 대화
4,4,협박 대화,1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...,공격 대화
...,...,...,...,...,...
4908,4908,일반 대화,NaN,시민의 교육파트너 입니다 무엇을 도와드릴까요\n예 초등돌봄교실 운영 관련해서 ...,일반 대화
4909,4909,일반 대화,NaN,전화 주신 곳은 상담 센터 입니다\n초등 단계로 입학 예정인데요\...,일반 대화
4910,4910,일반 대화,NaN,입니다\n안녕하세요 피시로 통합회원 하고 싶어서요\n고객님 기존에 사용하시던 간편 ...,일반 대화
4911,4911,일반 대화,NaN,화창한 아침입니다 입니다 무엇을 도와드릴까요\n에 대해 궁금한 점이 있...,일반 대화


# 토큰화
- Tensorflow Tokenizer사용
- 단어장 32000개
- 클래스 라벨 one-hot vector화
- train : valid : test = 0.7 : 0.15 : 0.15

In [55]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 32000

In [ ]:
sentences = data['spell_checked_conversation']

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(sentences)
word_dic = tokenizer.word_index
#print(word_dic)

sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

padded = pad_sequences(sequences)
print(padded)

In [57]:
np.shape(padded)

(4913, 291)

In [58]:
train_data = padded
train_label = data['class']

In [59]:
print(len(train_data), len(train_label))

4913 4913


In [60]:
pd.unique(train_label)

array(['직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '갈취 대화', '협박 대화', '일반 대화'],
      dtype=object)

In [63]:
labels = {'직장 내 괴롭힘 대화':0, '기타 괴롭힘 대화':1, '갈취 대화':2, '협박 대화':3, '일반 대화': 4}

In [64]:
train_label = train_label.apply(lambda x: labels[x])

In [65]:
train_label = pd.get_dummies(train_label)

In [66]:
print(train_label)

      0  1  2  3  4
0     1  0  0  0  0
1     0  1  0  0  0
2     0  0  1  0  0
3     0  0  1  0  0
4     0  0  0  1  0
...  .. .. .. .. ..
4908  0  0  0  0  1
4909  0  0  0  0  1
4910  0  0  0  0  1
4911  0  0  0  0  1
4912  0  0  0  0  1

[4913 rows x 5 columns]


In [67]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.3, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

In [68]:
print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

3439 737 737
3439 737 737


# 모델 
- LSTM 사용

In [76]:
word_vector_dim = 512
labels_size = len(labels)
hidden_size = 32

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size//2))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dense(labels_size, activation='softmax'))


model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 512)         16384000  
                                                                 
 lstm_21 (LSTM)              (None, None, 32)          69760     
                                                                 
 lstm_22 (LSTM)              (None, None, 32)          8320      
                                                                 
 lstm_23 (LSTM)              (None, 16)                3136      
                                                                 
 dense_16 (Dense)            (None, 32000)             544000    
                                                                 
 dense_17 (Dense)            (None, 5)                 160005    
                                                                 
Total params: 17,169,221
Trainable params: 17,169,221

In [101]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              
epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

In [80]:
pred = model.predict(test_X)

24/24 [==============================] - 4s 150ms/step


In [106]:
from sklearn.metrics import classification_report

pred_label = []
test_label = []

for i in range(len(pred)):
  pred_label.append((str)(np.argmax(pred[i])))
  test_label.append((str)(labels[data['class'][test_Y.index[i]]]))

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.78      0.75      0.77       155
           1       0.67      0.76      0.71       149
           2       0.85      0.70      0.77       146
           3       0.67      0.79      0.72       135
           4       1.00      0.92      0.96       152

    accuracy                           0.78       737
   macro avg       0.79      0.78      0.79       737
weighted avg       0.80      0.78      0.79       737

